In [1]:
{-# LANGUAGE InstanceSigs #-}
newtype State s a = State { runState :: s -> (a, s) }

In [2]:
instance Functor (State s) where
    fmap :: (a -> b) -> State s a -> State s b
    fmap f a = State $ \s -> let
        (a',s') = runState a s
        in (f a', s')

In [3]:
instance Applicative (State s) where
    pure :: a -> State s a
    pure a = State $ \s -> (a, s)
    (<*>) :: State s (a -> b) -> State s a -> State s b
    f <*> a = State $ \s -> let
        (f', s')  = runState f s
        (a', s'') = runState a s'
        in (f' a', s'')

Line 3: Use tuple-section
Found:
\ s -> (a, s)
Why not:
(a,)

In [4]:
instance Monad (State s) where
    (>>=) :: State s a -> (a -> State s b) -> State s b
    a >>= f = State $ \s -> let
        (a', s') = runState a s
        b        = f a'
        in runState b s'

In [5]:
newtype StateT s m a = StateT { runStateT :: s -> m (a, s) }

In [6]:
instance Functor m => Functor (StateT s m) where
  fmap :: (a -> b) -> StateT s m a -> StateT s m b
  fmap f (StateT statet) = StateT $ \s -> let
      statet' = statet s
      in (\(a, s') -> (f a, s')) <$> statet'

In [7]:
instance Monad m => Applicative (StateT s m) where
    pure :: a -> StateT s m a
    pure a = StateT $ \s -> pure (a, s)
    
    (<*>) :: StateT s m (a -> b) -> StateT s m a -> StateT s m b
    (<*>) (StateT mf) (StateT ma) = StateT $ \s -> do
        (f, s')  <- mf s
        (a, s'') <- ma s'
        pure (f a, s'')

In [8]:
instance Monad m => Monad (StateT s m) where
    (>>=) :: StateT s m a -> (a -> StateT s m b) -> StateT s m b
    (>>=) (StateT ma) f = StateT $ \s -> do
        (a, s') <- ma s
        let (StateT mb) = f a
        mb s'

In [9]:
import Control.Monad.Trans.Class

instance MonadTrans (StateT s) where
    lift :: Monad m => m a -> StateT s m a
    lift ma = StateT $ \s -> (\a -> (a, s)) <$> ma

Line 3: Use tuple-section
Found:
\ a -> (a, s)
Why not:
(, s)

In [27]:
get :: Monad m => StateT s m s
get = StateT $ \s -> pure (s, s)

put :: Monad m => s -> StateT s m ()
put s = StateT $ \_ -> pure ((), s)

In [34]:
guessing :: StateT (Maybe String) IO ()
guessing = do
    previousGuess <- get
    case previousGuess of
        Nothing -> go
        Just previousGuess -> do
            lift $ putStrLn $ "You previously guessed " ++ previousGuess
            go
    where
        go = do
            lift $ putStrLn "Guess my name: "
            guess <- lift $ getLine
            put (Just guess)
            if guess == "Vaibhav"
              then do
                  lift $ putStrLn "You got it right!"
              else do
                  guessing

runStateT guessing Nothing

Guess my name: 
You got it right!
((),Just "Vaibhav")